# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [25]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option")

for p in movie_item:
    try :
        print("Movie: %s, ID: %s" % (p['value'] , p["data-name"]))
    except :
        continue

Movie: 11348, ID: 猶大與黑色彌賽亞
Movie: 11338, ID: 真．鮫島事件
Movie: 11337, ID: 不日成婚
Movie: 11336, ID: 靈異詭店
Movie: 11324, ID: 我的袋鼠室友
Movie: 11317, ID: 搖滾農莊錄音趣
Movie: 11316, ID: 窒愛診療室
Movie: 11314, ID: 父親
Movie: 11313, ID: 逆轉棋蹟
Movie: 11309, ID: 電影版 嬌妻出沒注意
Movie: 11306, ID: 夢想之地
Movie: 11339, ID: 惡夜殺神
Movie: 11330, ID: 藥命交錯
Movie: 11325, ID: 暖心龍蝦湯
Movie: 11308, ID: 跟著奈良美智去旅行 經典重映
Movie: 11272, ID: 佳麗村三姊妹
Movie: 11249, ID: 樹海村
Movie: 11232, ID: 聽見歌 再唱
Movie: 11206, ID: 名偵探柯南：緋色的彈丸
Movie: 11073, ID: 永生戰
Movie: 11307, ID: 一場大火之後
Movie: 11303, ID: 郵差
Movie: 11288, ID: 歡樂時光〈後編〉
Movie: 11263, ID: 私刑救援
Movie: 11250, ID: 鐵幕行動
Movie: 11254, ID: 真人快打
Movie: 11297, ID: 衝鋒戰士：恐龍世代
Movie: 11286, ID: 男人與他的海
Movie: 11283, ID: 青春電幻物語：數位修復版
Movie: 11258, ID: 南巫
Movie: 11255, ID: 當男人戀愛時
Movie: 11126, ID: 無名弒
Movie: 11287, ID: 歡樂時光〈前編〉
Movie: 11277, ID: 波斯語課
Movie: 11270, ID: 令人討厭的松子的一生：15週年經典數位版
Movie: 11215, ID: 哥吉拉大戰金剛
Movie: 11221, ID: 游牧人生
Movie: 11241, ID: 今天決定我愛你
Movie: 11228, ID: 我愛你，你愛她，她愛他
Movie: 10519, ID

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [83]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 11348

In [77]:
import json
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/movietime_result.html?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'BX=0rr56t1g6qv3q&b=3&s=it; rxx=2ysdef5ebym.2aeymffm&v=1; _ga=GA1.4.1734044360.1617788031; __gads=ID=c1fc025d246d7775-2273321e08c70078:T=1617788030:RT=1617788030:S=ALNI_Mbj10zvLcqTYgy0Ly2FHN1cTPOFpg; A1=d=AQABBHp8bWACELxJLhppGV3dIjqgF-im7A0FEgEBBAHYh2BNYb3Lb2UB_eMAAAcIenxtYOim7A0&S=AQAAAoSTfOdAcRihJB1r9rczBvI; A3=d=AQABBHp8bWACELxJLhppGV3dIjqgF-im7A0FEgEBBAHYh2BNYb3Lb2UB_eMAAAcIenxtYOim7A0&S=AQAAAoSTfOdAcRihJB1r9rczBvI; A1S=d=AQABBHp8bWACELxJLhppGV3dIjqgF-im7A0FEgEBBAHYh2BNYb3Lb2UB_eMAAAcIenxtYOim7A0&S=AQAAAoSTfOdAcRihJB1r9rczBvI&j=WORLD; GUC=AQEBBAFgh9hhTUIdHwRN; _gid=GA1.4.1703003360.1619430521; avi=eyJpdiI6InV1c0Q3UlNOaFN6dEpqS0JWd3JXM1E9PSIsInZhbHVlIjoiaTg2alRkYUJPMWJ0a0c0c1hjeFMyNExLbW50NHlnMzBUV3RNREhhNGRjZFY2ZGdCTjk2Z1daT1VSd1VsR0poXC8iLCJtYWMiOiI5NzM4ZmFiYTdlYTRlYmM1YTgzNWNkNjgzMDA5MmVkNzdmNzQzOGU3NTdiNzNkYWU5MjU0NTlmNmM4YzM4YzA1In0%3D; cmp=t=1619434651&j=0; _gat=1; b_m2=eyJpdiI6ImorRTRmdUIxQmZYZ0tGaEQxMnNGY3c9PSIsInZhbHVlIjoiYzhReDRUTkZiekRwQUVDV3lzeTBuVXd3N3pXWlh6TWNZNEtmNzBhMzU1M1wvT0Q2aUwyRzlDT0VVRFVXNVhyZzZkMmNrSUtjR0hMNm5EaWdmMU4yU2FwR2x6UFNTVjJaQlJjbzVlYWtaRzlwTjVLSVdNRG93YU41S3hpSFlmbnRlZThoSHM1RnViK0FQZWRIcHdtNitEUT09IiwibWFjIjoiYjM1NjlmNzJhMDhiN2YzODUyYjMwMTkwOGY3YjZjZmY0OWM0MmY5YTEyZDU3YjFlNzY1MzRiZjhhYjM0NzdmYSJ9; XSRF-TOKEN=eyJpdiI6IkZWUGFwOHd1R3JKRnFxVjJmOW1kQVE9PSIsInZhbHVlIjoiSVJHdFBlQXpoVkFhTFBkM0h4UzlwaGRXYVpISTBjRnhXUUtQOGVET2RUSGh2WjJVWEMrVjB4amR4bmhwSHdoMiIsIm1hYyI6ImFiNWNiNDU1NjE0YTBmNTRmZTM3NDk3ODRmYjA1NWY4OTZmMDUyZDVkMjUzOTk2YmI2NWQyM2IwZmFmMWI4NWQifQ%3D%3D; ms55=eyJpdiI6ImlmYWFWam5kMnEyNnE1eU42aXpQdnc9PSIsInZhbHVlIjoiazhIU3ZcL3dZd3ZtN0trcXNFQTJKbEtqMWNhY2FkVXU5WWs1WVB4M1hiZm9lRkNJNkE1cGg1NTVRbTgyc3pndjFXYU5JSE94ZUs3dWJjc3h4SDhcL0dOV2o0Ymh4YVd1cFdCUDVCb1VyelF5U2V0UUp1VUo2dVdHZWlGSFZJZG56TiIsIm1hYyI6IjcxZTk3ZWM0OWNiODc5ZjcyMDI5N2QzZDE1M2EyMmZmNWEwYmE1MTA0YTdkZDQ1YmQ1ZjA5MmVjYzFiYjU2NDEifQ%3D%3D',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
}
    
resp = requests.get(url, params=payload, headers=headers)
print(resp.text)# 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
#for p in resp.json():
#   print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))
soup = BeautifulSoup(resp.text, 'lxml')

<!DOCTYPE html>
<html lang="zh_TW">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1">
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta property="fb:app_id" content="501887343352051">
  <meta property="og:site_name" content="Yahoo奇摩電影">
    <title>猶大與黑色彌賽亞 Judas and the Black Messiah電影時刻表 - Yahoo奇摩電影</title>
<meta name="twitter:title" content="猶大與黑色彌賽亞 Judas and the Black Messiah電影時刻表 - Yahoo奇摩電影">
  <meta property="og:title" content="猶大與黑色彌賽亞 Judas and the Black Messiah電影時刻表 - Yahoo奇摩電影">

<link rel="canonical" href="https://movies.yahoo.com.tw/movietime_result.html?movie_id=11348">
<meta property="og:url" content="https://movies.yahoo.com.tw/movietime_result.html?movie_id=11348">

<meta name="twitter:description" content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態">
<meta name="description" content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態">

  <me

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [84]:
# 指定放映地區
area_id = 28

In [85]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

四月 27
四月 28
四月 29
四月 30
五月 1


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [96]:
# 選定要觀看的日期
date = "2021-04-28"

In [102]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           #'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

<div class="pc-movie-schedule-form">
    <div class="area_timebox">
   <div class="area_title">台北市</div>
      <ul id="theater_id_32" class="area_time _c jq_area_time" data-theater_name="台北美麗華大直影城" data-theater_url="http://www.miramarcinemas.tw/" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=32" >
       <li class="adds">
           <a href="https://movies.yahoo.com.tw/theater_result.html/id=32">台北美麗華大直影城</a>
           <span>02-85022208</span>
       </li>
                         <li class="taps">
                <span class="tapR">數位</span>                           </li>
           <li class="time _c">
               <div class="input_picker jq_input_picker">
                                                                   <input type="radio" name="schedule_list" id="75348724" class="gabtn"
                        value="2021-04-28 11:40:00"  data-movie="c7474c7552ea4b01b75edbbfff034028"
                        data-cinema="40288eb0050f31210ada0f55676

In [122]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
cities = soup.find_all('div' , attrs={'class':'area_timebox'})
name = soup.find_all('ul',attrs = {"data-theater_name":re.compile(".*")})
for c in cities :
    print(c.find(class_="area_title").text)
    print()
    for d in c.find_all('ul',attrs = {"data-theater_name":re.compile(".*")}):
        print("影廳:" ,d["data-theater_name"]) 
        for time in d.find_all('input') :
            print("類型:" ,time["data-movie_type"], "時間:" ,time["data-movie_date"] ,time["data-movie_time"])
        print('\n')
    print("\n")

台北市

影廳: 台北美麗華大直影城
類型: 數位 時間: 04.28 11:40
類型: 數位 時間: 04.28 16:25
類型: 數位 時間: 04.28 20:45


影廳: 台北真善美劇院
類型: 數位 時間: 04.28 14:30
類型: 數位 時間: 04.28 19:00


影廳: 台北信義威秀影城
類型: 數位 時間: 04.28 12:15
類型: 數位 時間: 04.28 16:40
類型: 數位 時間: 04.28 19:05
類型: 數位 時間: 04.29 00:55


影廳: 京站威秀影城
類型: 數位 時間: 04.28 10:40
類型: 數位 時間: 04.28 15:10
類型: 數位 時間: 04.28 19:25


影廳: 喜樂時代影城南港店
類型: 數位 時間: 04.28 13:00
類型: 數位 時間: 04.28 15:20
類型: 數位 時間: 04.28 19:25
類型: 數位 時間: 04.28 21:45


影廳: MUVIE CINEMAS台北松仁威秀
類型: 數位 時間: 04.28 13:25
類型: 數位 時間: 04.28 21:50




新北市

影廳: 板橋大遠百威秀影城
類型: 數位 時間: 04.28 12:20
類型: 數位 時間: 04.28 16:40
類型: 數位 時間: 04.28 21:30




桃園

影廳: 桃園統領威秀影城
類型: 數位 時間: 04.28 10:30
類型: 數位 時間: 04.28 12:50
類型: 數位 時間: 04.28 15:20
類型: 數位 時間: 04.28 17:40
類型: 數位 時間: 04.28 20:00
類型: 數位 時間: 04.28 22:20




新竹

影廳: 新竹巨城威秀影城
類型: 數位 時間: 04.28 11:00
類型: 數位 時間: 04.28 15:55
類型: 數位 時間: 04.28 20:25




台中

影廳: 台中Tiger City威秀影城
類型: 數位 時間: 04.28 11:30
類型: 數位 時間: 04.28 15:10
類型: 數位 時間: 04.28 20:55




台南

影廳: 台南南紡威秀影城
類型: 數位 時間: 04.28 11:10
